#### Note: This will run if you have the the movies saved as "start_numor".npz files. Check other notebook titled SANS_to_npz.ipynb for code to assist in doing this.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter
from scipy.signal import argrelextrema
from ipywidgets import interact, IntSlider
from scipy.signal import argrelextrema

In [ ]:
folder_movies = {
    # field sweeps
    '55.3 K 50 mW Negative Field Sweep': ['54261', '54863', '55162', '55461', '55760', '56059', '56358'],
    '55.3 K 50 mW Positive Field Sweep': ['73131', '73430', '73729', '74028', '74327', '74626'],
    '55.8 K 25 mW Negative Field Sweep': ['68463', '68762', '69061', '69360', '69659', '69958'],
    '55.8 K 25 mW Positive Field Sweep': ['70797', '71096', '71395', '71694', '71993', '72292'],

    # temperature sweeps (only including those with signal)
    '-29 mT 10 mW Temp Sweep': ['61221', '61759', '62417'],
    '-29 mT 25 mW Temp Sweep': ['64690', '64152', '63614', '63076'],
    '+29 mT 25 mW Temp Sweep': ['76899', '76361', '75823', '75285'],
    '-29 mT 50 mW Temp Sweep': ['52666', '59887', '52128', '59349', '51590', '58811', '51052', '58273', '50514', '57735', '49976', '57197', '49438'],
    '+29 mT 50 mW Temp Sweep': ['47963', '47425', '67624', '46887', '67086', '46349', '45811', '66670', '45268', '44730', '44192']
}
# bad data
# '-29 mT 25 mW Temp Sweep': 65766, 65228, 66304


# Movie numor matched with each title
movie_to_title = {
    # --- Field Sweeps ---
    
    # 55.3 K 50 mW Negative Field Sweep
    '54261': '54261 55.3 K -23 mT 50 mW',
    '54863': '54863 55.3 K -25 mT 50 mW',
    '55162': '55162 55.3 K -27 mT 50 mW',
    '55461': '55461 55.3 K -29 mT 50 mW',
    '55760': '55760 55.3 K -31 mT 50 mW',
    '56059': '56059 55.3 K -33 mT 50 mW',
    '56358': '56358 55.3 K -35 mT 50 mW',

    # 55.3 K 50 mW Positive Field Sweep
    '73131': '73131 55.3 K +23 mT 50 mW',
    '73430': '73430 55.3 K +25 mT 50 mW',
    '73729': '73729 55.3 K +27 mT 50 mW',
    '74028': '74028 55.3 K +29 mT 50 mW',
    '74327': '74327 55.3 K +31 mT 50 mW',
    '74626': '74626 55.3 K +33 mT 50 mW',

    # 55.8 K 25 mW Negative Field Sweep
    '68463': '68463 55.8 K -23 mT 25 mW',
    '68762': '68762 55.8 K -25 mT 25 mW',
    '69061': '69061 55.8 K -27 mT 25 mW',
    '69360': '69360 55.8 K -29 mT 25 mW',
    '69659': '69659 55.8 K -31 mT 25 mW',
    '69958': '69958 55.8 K -33 mT 25 mW',

    # 55.8 K 25 mW Positive Field Sweep
    '70797': '70797 55.8 K +23 mT 25 mW',
    '71096': '71096 55.8 K +25 mT 25 mW',
    '71395': '71395 55.8 K +27 mT 25 mW',
    '71694': '71694 55.8 K +29 mT 25 mW',
    '71993': '71993 55.8 K +31 mT 25 mW',
    '72292': '72292 55.8 K +33 mT 25 mW',

    # --- Temperature Sweeps ---

    # -29 mT 10 mW Temp Sweep
    '61221': '61221 56.0 K -29 mT 10 mW',
    '61759': '61759 56.3 K -29 mT 10 mW',
    '62417': '62417 57.5 K -29 mT 10 mW',

    # -29 mT 25 mW Temp Sweep
    '65766': '65766 54.8 K -29 mT 25 mW',
    '65228': '65228 55.3 K -29 mT 25 mW',
    '64690': '64690 55.8 K -29 mT 25 mW',
    '64152': '64152 56.3 K -29 mT 25 mW',
    '63614': '63614 56.8 K -29 mT 25 mW',
    '63076': '63076 57.3 K -29 mT 25 mW',

    # +29 mT 25 mW Temp Sweep
    '76899': '76899 55.8 K +29 mT 25 mW',
    '76361': '76361 56.3 K +29 mT 25 mW',
    '75823': '75823 56.8 K +29 mT 25 mW',
    '75285': '75285 57.3 K +29 mT 25 mW',

    # -29 mT 50 mW Temp Sweep
    '52666': '52666 54.3 K -29 mT 50 mW',
    '59887': '59887 54.5 K -29 mT 50 mW',
    '52128': '52128 54.8 K -29 mT 50 mW',
    '59349': '59349 55.0 K -29 mT 50 mW',
    '51590': '51590 55.3 K -29 mT 50 mW',
    '58811': '58811 55.5 K -29 mT 50 mW',
    '51052': '51052 55.8 K -29 mT 50 mW',
    '58273': '58273 56.0 K -29 mT 50 mW',
    '50514': '50514 56.3 K -29 mT 50 mW',
    '57735': '57735 56.5 K -29 mT 50 mW',
    '49976': '49976 56.8 K -29 mT 50 mW',
    '57197': '57197 57.0 K -29 mT 50 mW',
    '49438': '49438 57.3 K -29 mT 50 mW',

    # +29 mT 50 mW Temp Sweep
    '47963': '47963 53.8 K +29 mT 50 mW',
    '47425': '47425 54.3 K +29 mT 50 mW',
    '67624': '67624 54.5 K +29 mT 50 mW',
    '46887': '46887 54.8 K +29 mT 50 mW',
    '67086': '67086 55.0 K +29 mT 50 mW',
    '46349': '46349 55.3 K +29 mT 50 mW',
    '45811': '45811 55.8 K +29 mT 50 mW',
    '66670': '66670 55.9 K +29 mT 50 mW',
    '45268': '45268 56.3 K +29 mT 50 mW',
    '44730': '44730 56.8 K +29 mT 50 mW',
    '44192': '44192 57.3 K +29 mT 50 mW',
}
[46349, 69958, 56059]
filtered_indices = {
    # --- Field Sweeps ---
    
    # 55.3 K 50 mW Negative Field Sweep
    '54261': [],
    '54863': [], 
    '55162': [], 
    '55461': [15, 16],
    '55760': [], 
    '56059': [], 
    '56358': [],

    # 55.3 K 50 mW Positive Field Sweep
    '73131': [7], 
    '73430': [], 
    '73729': [], 
    '74028': [], 
    '74327': [129, 130, 214, 215], 
    '74626': [],

    # 55.8 K 25 mW Negative Field Sweep
    '68463': [265, 266],
    '68762': [39, 40, 175, 176, 206],
    '69061': [], 
    '69360': [254, 255], 
    '69659': [132, 133, 252, 253], 
    '69958': [271, 272],

    # 55.8 K 25 mW Positive Field Sweep
    '70797': [47, 48, 216, 265, 266], 
    '71096': [], 
    '71395': [], 
    '71694': [35, 36], 
    '71993': [265, 266],
    '72292': [],

    # --- Temperature Sweeps ---
    
    # -29 mT 10 mW Temp Sweep
    '61221': [], 
    '61759': [], 
    '62417': [],

    # -29 mT 25 mW Temp Sweep
    '65766': [], 
    '65228': [], 
    '64690': [], 
    '64152': [], 
    '63614': [], 
    '63076': [],

    # +29 mT 25 mW Temp Sweep
    '76899': [], 
    '76361': [], 
    '75823': [], 
    '75285': [],

    # -29 mT 50 mW Temp Sweep
    '52666': [], 
    '59887': [], 
    '52128': [], 
    '59349': [], 
    '51590': [], 
    '58811': [], 
    '51052': [], 
    '58273': [], 
    '50514': [], 
    '57735': [], 
    '49976': [], 
    '57197': [], 
    '49438': [],

    # +29 mT 50 mW Temp Sweep
    '47963': [], 
    '47425': [], 
    '67624': [], 
    '46887': [], 
    '67086': [], 
    '46349': [], 
    '45811': [], 
    '66670': [], 
    '45268': [], 
    '44730': [], 
    '44192': []
}

print(filtered_indices.keys())


In [ ]:
# "easy": 69958, "med": 56059, "hard": 46349
########### REFERENCE FRAME ##############
#   0deg is the +y axis
#   postive phi is cw rotation
#   ccw rotation caused by negative field
#   cw rotation caused by positive field
#   TLDR: positive fields have positive phis and negative fields have negative phis

numor = 69958
path_to_npz = f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/npz_sept_numor_data/{numor}.npz'
data = np.load(path_to_npz)['data']
print(data[0].shape)
print(movie_to_title[str(numor)])
n_folds=6
h, w = data[0].shape[:2]
cx, cy = w // 2, h // 2
x_grid, y_grid = np.meshgrid(np.arange(w), np.arange(h))
# handle slight offcentering of DPs
shift_x, shift_y = 0, -2
x_grid_shifted = x_grid - shift_x
y_grid_shifted = y_grid - shift_y
DATA_THETA = np.arctan2(y_grid_shifted - cy, x_grid_shifted - cx) # switch to arctan(x/y) for reference frame
OFFSET_ADJUSTMENT = int(360/n_folds)
OFFSET_ADJUSTMENT_rad = np.deg2rad(OFFSET_ADJUSTMENT)
resolution=10.8

def find_k_value(resolution=resolution, n_folds=n_folds):
    '''finds k given resolution(deg).
    This assumes FWHM_experiment is approx equal to FWHM_filter'''
    res_rad = np.deg2rad(resolution)
    k = np.log(1/2) / (np.log(np.cos((n_folds/4*res_rad))**2))
    # print('k=', k)
    return k
k = find_k_value()

def filter_function(k, offset, n_folds=n_folds):
    filter = np.exp(k * np.log((np.cos(n_folds / 2 * (DATA_THETA + offset)))**2)) # changed cosine to sine for reference frame
    # plt.imshow(filter)
    # plt.title(f'n_folds={n_folds}, k={k}')
    # plt.show()
    return filter

def normalize_min_max(data):
    try:
        array = data.copy()
    except AttributeError:
        array = np.array(data, copy=True)
    if array.size == 0:
        raise ValueError("Cannot normalize an empty array.")
    array_min = np.min(array)
    array_max = np.max(array)
    if array_max == array_min:
        return np.zeros_like(array)
    norm_array = (array - array_min) / (array_max - array_min)
    return norm_array

def mask_and_blur_images(movie, inner_radius=16, outer_radius=30, sigma=0.65):
    for i in range(movie.shape[0]):
        x, y = np.meshgrid(np.arange(128), np.arange(128))
        radius = np.sqrt((x - 64)**2 + (y - 62)**2)
        mask1 = radius <= inner_radius
        mask2 = radius >= outer_radius
        masked_data = movie[i].copy()
        masked_data[mask1] = 0
        masked_data2 = masked_data.copy()
        masked_data2[mask2] = 0
        blurred_data = gaussian_filter(masked_data2, sigma=sigma)
        movie[i] = blurred_data
    return movie

dps = mask_and_blur_images(data)
# plt.imshow(normalize_min_max(dps[0]) + filter_function(200, 0, n_folds=2))


## Grid Search

In [ ]:
### SECOND DERIVATIVE FUNCTIONS AND SUCH

# Function 2: get correct period indices
def get_correct_period_indices(loss_values):
    """Finds the correct period indices based on the maximum value in the loss values.
    If only one max index is found, adds adjacent indices at -60 or +60."""
    indices = [i for i, loss in enumerate(loss_values) if loss == max(loss_values)]
    if len(indices) > 1:
        return sorted(indices[:2])
    index = indices[0]
    adjacent_indices = [i for i in [index + OFFSET_ADJUSTMENT, index - OFFSET_ADJUSTMENT] if 0 <= i < len(loss_values)]
    return sorted([index] + adjacent_indices)

# Function 3: Discrete search for number of domains and get approximate offset positions
def compute_loss_near_offset1(intensity, prev_offset):
    """Compute the loss, first derivative, and second derivative for a range of offsets
    around the previous offset, selecting the closest periodic range.

    prev_offset should be in radians"""
    
    # Define the periodic range and step size for sweeping through offset angles
    angle_period = np.deg2rad(360/n_folds)
    step_size_radians = np.deg2rad(1)
    offset_values = np.arange(prev_offset-angle_period, prev_offset+angle_period, step_size_radians)
    loss_values = []
    for offset in offset_values:
        evaluate_image_theta = filter_function(k, offset)
        loss = -(intensity * evaluate_image_theta).sum()
        loss_values.append(loss.item())
    first_derivative = np.gradient(loss_values)
    second_derivative = np.gradient(first_derivative)
    selected_indices = get_correct_period_indices(loss_values)

    # Select the appropriate range of offsets, losses, and derivatives based on the period indices
    offset_values = np.array(offset_values[selected_indices[0]:selected_indices[1]+1])
    loss_values = np.array(loss_values[selected_indices[0]:selected_indices[1]+1])
    first_derivative = np.array(first_derivative[selected_indices[0]:selected_indices[1]+1])
    second_derivative = np.array(second_derivative[selected_indices[0]:selected_indices[1]+1])

    return offset_values, loss_values, first_derivative, second_derivative

# Function 4: Get approximate offset positions through finding local maxima in second derivatives
def get_approx_offset_values(offset_values, second_derivative):
    """Identify approximate offset positions by finding local maxima and inflection points in the second derivative."""
    local_maxima_indices = argrelextrema(second_derivative, np.greater)[0]
    # Remove indices close to the edges because these local maximas might be introduced by the local maxima from the phase change
    filtered_indices = local_maxima_indices[(local_maxima_indices >= 5) & (local_maxima_indices <= len(second_derivative) - 6)]
    indices_info = [(index, offset_values[index], second_derivative[index]) for index in filtered_indices]
    # Sort by second derivative values (descending) and return top two offsets
    sorted_maxima = sorted(indices_info, key=lambda x: x[2], reverse=True)
    approx_offset_values = [offset for _, offset, _ in sorted_maxima][:2]
    return approx_offset_values

In [ ]:
# STORING FOR BACKUP, DELETE LATER
def grid_search_offset(
    dps,
    tolerance_forward,
    tolerance_reverse,
    jump_threshold_offset1,
    jump_threshold_offset2,
    offset_step=0.5,
):
    '''
    positive angles = forward rotation
    negative angles = reverse rotation
    returns offset1 and offset2 in degrees
    '''
    optimal_offsets1 = []
    optimal_offsets2 = []
    prev_offset1 = 0
    prev_offset2 = 0

    for index, dp in enumerate(dps):
        loss_list1 = []
        offset_list_deg1 = []
        # find initial offset
        if index == 0:
            offset1_angles_deg = np.arange(-60, 60, offset_step)
        else:
            offset1_angles_deg = np.arange(prev_offset1 - tolerance_reverse, prev_offset1 + tolerance_forward, offset_step)

        # Compute loss function
        for offset1 in offset1_angles_deg:
            offset_rad = np.deg2rad(offset1)  # convert to radians
            filt = filter_function(k, offset_rad)
            loss = -(dp * filt).sum()
            loss_list1.append(loss)
            offset_list_deg1.append(offset1)
            if offset1 % 5 ==0:
                print(offset1)
                plt.imshow(filt + normalize_min_max(dp))
                plt.show()
        min_loss_idx = loss_list1.index(min(loss_list1))
        best_offset1 = offset_list_deg1[min_loss_idx]
        # Convert to NumPy arrays
        loss_array = np.array(loss_list1)
        offset_array = np.array(offset_list_deg1)
        # Global argmax
        global_argmax_idx = np.argmax(loss_array)
        global_argmax_offset = offset_array[global_argmax_idx]

        # Local argmax
        local_argmax_indices = argrelextrema(loss_array, np.greater)[0]
        local_argmax_offsets = offset_array[local_argmax_indices]

        # print(f"{index}: Global argmax: {global_argmax_offset}, Local argmax: {local_argmax_offsets}")

        # Correct overrotations of 60 degrees for offset1
        delta1 = abs(global_argmax_offset - prev_offset1)
        if delta1 > jump_threshold_offset1 and len(local_argmax_offsets) > 0:
            best_offset1 = min(local_argmax_offsets, key=lambda x: abs(x - prev_offset1))
            # print(f"{index}: Threshold exceeded for offset1. Using closest local max: {best_offset1}")
        else:
            best_offset1 = global_argmax_offset

        filt1 = filter_function(k, np.deg2rad(best_offset1))
        filt1 = np.where(filt1 > 0.01, 0, 1)
        dp_filtered = dp * filt1

        # Use second derivative to determine the number of domains
        offset_range, loss_values, first_derivative, second_derivative = compute_loss_near_offset1(dp, np.deg2rad(best_offset1))
        approx_offset_values = get_approx_offset_values(offset_range, second_derivative)
        number_of_domains = len(approx_offset_values)
        loss_list2 = []
        offset_list_deg2 = []
        if number_of_domains == 1:
            offset2_angles_deg = np.arange(prev_offset2 - 40, prev_offset2 + 40, offset_step)
            for offset2 in offset2_angles_deg:
                offset_rad = np.deg2rad(offset2)
                filt2 = filter_function(k, offset_rad)
                loss = -(dp * filt2).sum()
                loss_list2.append(loss)
                offset_list_deg2.append(offset2)
            min_loss_idx = loss_list2.index(min(loss_list2))
            best_offset2 = offset_list_deg2[min_loss_idx]
        else:
            # Second domain search
            offset2_angles_deg = np.arange(prev_offset2 - tolerance_reverse, prev_offset2 + tolerance_forward, offset_step)
            for offset2 in offset2_angles_deg:
                offset_rad = np.deg2rad(offset2)
                filt2 = filter_function(k, offset_rad)
                loss = -(dp_filtered * filt2).sum()
                loss_list2.append(loss)
                offset_list_deg2.append(offset2)
            min_loss_idx = loss_list2.index(min(loss_list2))
            best_offset2 = offset_list_deg2[min_loss_idx]

            # Correct overrotations of 60 degrees for offset2
            delta2 = abs(best_offset2) - abs(prev_offset2)
            if delta2 > jump_threshold_offset2:
                best_offset2 -= 60
                print(f"{index}: Threshold2 exceeded for offset2. Adjusting by -60 degrees: {best_offset2}")
                print(delta2, '>', jump_threshold_offset2)

        # plotting
        norm_int = normalize_min_max(dp)
        filt1 = filter_function(k, np.deg2rad(best_offset1))
        filt2 = filter_function(k, np.deg2rad(best_offset2))
        signal = norm_int + filt1 + 2*filt2
        fig, axs = plt.subplots(1, 3, figsize=(12, 6))
        axs[0].imshow(signal, cmap='viridis', vmin=0, vmax=2)
        axs[1].imshow(norm_int, cmap='viridis')
        axs[2].imshow(dp_filtered)
        plt.show()
        print(f'{index*10}s: ', best_offset1, best_offset2)

        optimal_offsets1.append(best_offset1)
        optimal_offsets2.append(best_offset2)
        prev_offset1 = best_offset1
        prev_offset2 = best_offset2
    return optimal_offsets1, optimal_offsets2

tolerance_forward = 20
tolerance_reverse = 5
jump_threshold_offset1 = 20
jump_threshold_offset2 = 20
offset1, offset2 = grid_search_offset(dps[:2], offset_step=0.5, tolerance_forward=tolerance_forward, tolerance_reverse=tolerance_reverse, 
                                        jump_threshold_offset1=jump_threshold_offset1, jump_threshold_offset2=jump_threshold_offset2)


In [ ]:

def grid_search_offset(
    dps,
    tolerance_forward,
    tolerance_reverse,
    jump_threshold_offset1,
    jump_threshold_offset2,
    n_folds=6,
    offset_step=0.5,
):
    '''
    positive angles = cw rotation
    negative angles = ccw rotation
    returns offset1 and offset2 in degrees
    '''
    optimal_offsets1 = []
    optimal_offsets2 = []
    prev_offset1 = 0
    prev_offset2 = 0

    for index, dp in enumerate(dps):
        loss_list1 = []
        offset_list_deg1 = []
        # find initial offset
        if index == 0:
            offset1_angles_deg = np.arange(-360/n_folds, 360/n_folds, offset_step)
        else:
            offset1_angles_deg = np.arange(prev_offset1 - tolerance_reverse, prev_offset1 + tolerance_forward, offset_step)
        ######## CALCULATE FIRST OFFSET #########
        # Compute loss function
        for offset1 in offset1_angles_deg:
            offset_rad = np.deg2rad(offset1)  # convert to radians
            filt = filter_function(k, offset_rad)
            loss = -(dp * filt).sum()
            loss_list1.append(loss)
            offset_list_deg1.append(offset1)
        # Convert to NumPy arrays
        loss_array = np.array(loss_list1)
        offset_array = np.array(offset_list_deg1)
        # Global argmax
        global_argmax_idx = np.argmax(loss_array)
        global_argmax_offset = offset_array[global_argmax_idx]
        # Local argmax
        local_argmax_indices = argrelextrema(loss_array, np.greater)[0]
        local_argmax_offsets = offset_array[local_argmax_indices]
        # print(f"{index}: Global argmax: {global_argmax_offset}, Local argmax: {local_argmax_offsets}")

        # Correct overrotations of 60 degrees for offset1
        delta1 = abs(global_argmax_offset - prev_offset1)
        if delta1 > jump_threshold_offset1 and len(local_argmax_offsets) > 0:
            best_offset1 = min(local_argmax_offsets, key=lambda x: abs(x - prev_offset1))
            # print(f"{index}: Threshold exceeded for offset1. Using closest local max: {best_offset1}")
        else:
            best_offset1 = global_argmax_offset

        filt1 = filter_function(k, np.deg2rad(best_offset1))
        filt1 = np.where(filt1 > 0.05, 0, 1)
        dp_filtered = dp * filt1

        ######### CALCULATE SECOND OFFSET #########
        # Use second derivative to determine the number of domains
        offset_range, loss_values, first_derivative, second_derivative = compute_loss_near_offset1(dp, np.deg2rad(best_offset1))
        approx_offset_values = get_approx_offset_values(offset_range, second_derivative)
        number_of_domains = len(approx_offset_values)
        if number_of_domains == 1:
            best_offset2 = prev_offset2 + best_offset1 % 360/n_folds  # mod60 term is added to account for the
        else:                                               # movement of offset2 from previous to current frame
            # Second domain search
            loss_list2 = []
            offset_list_deg2 = []
            offset2_angles_deg = np.arange(prev_offset2 - tolerance_reverse, prev_offset2 + tolerance_forward, offset_step)
            for offset2 in offset2_angles_deg:
                offset_rad = np.deg2rad(offset2)
                filt2 = filter_function(k, offset_rad)
                loss = -(dp_filtered * filt2).sum()
                loss_list2.append(loss)
                offset_list_deg2.append(offset2)
            min_loss_idx = loss_list2.index(min(loss_list2))
            best_offset2 = offset_list_deg2[min_loss_idx]

            # Correct overrotations of 60 degrees for offset2
            delta2 = abs(best_offset2) - abs(prev_offset2)
            # print(f"{index}: best_offset2 = {best_offset2}, prev_offset2 = {prev_offset2}, delta2 = {delta2}")
            if delta2 > jump_threshold_offset2:
                best_offset2 -= 360/n_folds
                print(f"{index}: Threshold2 exceeded for offset2. Adjusting by -{360/n_folds} degrees: {best_offset2}")
                print(delta2, '>', jump_threshold_offset2)

        # plotting
        # norm_int = normalize_min_max(dp)
        # filt1 = filter_function(k, np.deg2rad(best_offset1))
        # filt2 = filter_function(k, np.deg2rad(best_offset2))
        # signal = norm_int + filt1 + 2*filt2
        # fig, axs = plt.subplots(1, 3, figsize=(12, 6))
        # axs[0].imshow(signal, cmap='viridis', vmin=0, vmax=2)
        # axs[1].imshow(norm_int, cmap='viridis')
        # axs[2].imshow(dp_filtered)
        # plt.show()
        # print(f'{index*10}s: ', best_offset1, best_offset2)

        optimal_offsets1.append(best_offset1)
        optimal_offsets2.append(best_offset2)
        prev_offset1 = best_offset1
        prev_offset2 = best_offset2
    return optimal_offsets1, optimal_offsets2


# Fix jump thresholding and then fix the new frame of reference
tolerance_forward = 20
tolerance_reverse = 5
jump_threshold_offset1 = 20
jump_threshold_offset2 = 20
offset1, offset2 = grid_search_offset(dps, offset_step=0.5, tolerance_forward=tolerance_reverse, tolerance_reverse=tolerance_forward, 
                                        jump_threshold_offset1=jump_threshold_offset1, jump_threshold_offset2=jump_threshold_offset2)


In [ ]:
# TEST BELOW FOR LOOP ON INDIVIDUAL SWEEPS
path_to_npzs = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/npz_sept_numor_data/'

tolerance_forward = 20
jump_threshold_offset1 = 20
jump_threshold_offset2 = 20

def process_numors(numors, tolerance_reverse, filtered_indices=filtered_indices, tolerance_forward=tolerance_forward, 
                   jump_threshold_offset1=jump_threshold_offset1, jump_threshold_offset2=jump_threshold_offset2, 
                   path_to_npzs=path_to_npzs):
    """Processes a list of numors, removes bad DPs, applies masking, and calculates offsets.
    Returns:
        offsets_info (dict): {numor: {"offset1": np.array, "offset2": np.array}}
    """
    offsets_info = {}
    print(f'Processing {len(numors)} numors...')
    print(f'Reverse tolerance= {tolerance_reverse} deg')

    original_tolerance_forward = tolerance_forward
    original_tolerance_reverse = tolerance_reverse
    for numor in numors:
        # logic for cw (pos) or ccw (neg) skyrmion motion, ccw is default
        if '+' in movie_to_title[str(numor)] or 'Positive' in movie_to_title[str(numor)]:
            tolerance_forward, tolerance_reverse = tolerance_reverse, tolerance_forward
            print('   Positive field detected, swapping tolerances -->', movie_to_title[str(numor)])
            print('   tolerance_forward <--> tolerance_reverse')

        print(f'Processing numor: {numor}')
        data = np.load(f"{path_to_npzs}{numor}.npz")['data']
        # Remove bad DPs if numor exists in filtered_indices
        if numor in filtered_indices:
            for index in sorted(filtered_indices[numor], reverse=True):  # Reverse to avoid index shifting issues
                print(f'Deleting index {index} from {numor}')
                data = np.delete(data, index, axis=0)
        dps = mask_and_blur_images(data)
        offsets1, offsets2 = grid_search_offset(dps, offset_step=0.5, tolerance_forward=tolerance_forward, 
                                                tolerance_reverse=tolerance_reverse, jump_threshold_offset1=jump_threshold_offset1, 
                                                jump_threshold_offset2=jump_threshold_offset2)
        offsets_info[numor] = {"offset1": offsets1, "offset2": offsets2}
        tolerance_forward, tolerance_reverse = original_tolerance_forward, original_tolerance_reverse
    print('Processing complete.')
    return offsets_info  # Dictionary: {numor: {"offset1": array, "offset2": array}}


def plot_numor_offsets(offsets_info, tolerance_reverse, tolerance_forward=tolerance_forward, 
                       jump_threshold_offset1=jump_threshold_offset1, jump_threshold_offset2=jump_threshold_offset2):
    """Plots offsets for all processed numors."""
    numors = list(offsets_info.keys())
    colors = [plt.cm.viridis(i / len(numors)) for i in range(len(numors))]
    print("Generating plots...")
    for i, (numor, offsets) in enumerate(offsets_info.items()):
        offset1, offset2 = offsets["offset1"], offsets["offset2"]
        plt.plot(offset1 - offset1[0], label=f'offset1 {numor}', color=colors[i])
        plt.plot(offset2 - offset2[0], color=colors[i], linestyle='dotted')
    plt.title(f"tolerance_forward={tolerance_forward}, tolerance_reverse={tolerance_reverse}\n"
              f"jump_threshold1={jump_threshold_offset1}, jump_threshold2={jump_threshold_offset2}")
    plt.xlabel("Index")
    plt.ylabel("Offset (degrees)")
    plt.minorticks_on()
    plt.tick_params(direction='in')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'data/reverse_tol_{tolerance_reverse}_numors_ratchet_model.png')
    plt.show()



In [ ]:

dataset = '55.3 K 50 mW Negative Field Sweep'
# numors = folder_movies[dataset]
numors = [46349, 69958, 56059] #pos, neg, neg
print(numors)
tols = [8, 7, 6, 5, 4, 3]
for tol_reverse in tols:
    print(f'tol_reverse={tol_reverse}')
    offset_info = process_numors(numors, tol_reverse)
    for numor, offsets in offset_info.items():
        np.savez(f"data/reverse_tol_{tol_reverse}_{numor}_offsets.npz", 
                 **{f"offset1": offsets["offset1"], 
                    f"offset2": offsets["offset2"]})
    plot_numor_offsets(offset_info, tol_reverse)





## Interactive plotting

In [ ]:
numor = '61221'
path_to_offsets = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/ratchet_model/data/'
filename = f'ratchet_offsets_{numor}.npz'
offset1 = -np.load(path_to_offsets + filename)['offset1']
offset2 = -np.load(path_to_offsets + filename)['offset2']

# offsets1 = np.rad2deg(offsets1)
# print(offsets1)
# offsets2 = np.deg2rad(np.array(offset2))
# print(offset1[:5])
# print(offset2[:5])
# offsets2 = np.rad2deg(offsets2)
# print(offsets2)

def interactive_plot(frame_idx):
    '''Plot the intensity data and filter images with respect to the selected frame index.'''
    offset1_rad = np.deg2rad(offset1[frame_idx])
    offset2_rad = np.deg2rad(offset2[frame_idx])
    n_folds = 6
    k=find_k_value(n_folds=n_folds)
    image1 = filter_function(k, offset1_rad, n_folds=n_folds)
    image2 = filter_function(k, offset2_rad, n_folds=n_folds)
    dp_norm = normalize_min_max(dps[frame_idx])
    # Create figure and axes
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))

    # first domain
    ax1.imshow(image1 + dp_norm)
    # ax1.imshow(image1 + 2*image2 + normalize_min_max(dps[frame_idx]), origin="lower")
    ax1.set_title(f"Frame {frame_idx+1}: first domain")
    ax1.axis("off")

    # second domain
    ax2.imshow(image2 + dp_norm)
    ax2.set_title("2nd domain")

    # raw DP
    ax3.imshow(dp_norm)
    ax3.set_title(f"Frame {frame_idx+1}: Intensity Data")
    ax3.axis("off")
    # Update the figure title with offset values
    fig.suptitle(f"numor={numor}, \
                 Offset1={round(offset1[frame_idx], 2)}°, \
                Offset2={round(offset2[frame_idx], 2)}°, \
                Time={(frame_idx+1)*10}s, \
                Reverse tol={tolerance_reverse}")
    
    # Display the plot
    plt.tight_layout()
    plt.show()

# Create interactive plot with sliders for frame index
interact(
    interactive_plot,
    frame_idx=IntSlider(value=0, min=0, max=len(dps)-1, step=1, description='Frame Index')
);



## Run ratchet on all data

In [ ]:
import matplotlib.cm as cm
cmap = cm.viridis

# neg field 63614 moves ccw
# pos field 75823 moves cw

# tolerance_ccw=40
# tolerance_cw=5
jump_threshold_offset1 = 15
jump_threshold_offset2 = 50
path_to_npzs = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/npz_sept_numor_data/'
for title, numor_list in folder_movies.items():
    # Initialize a list to store results for plotting
    all_offsets = []
    colors = [cmap(i / len(numor_list)) for i in range(len(numor_list))]
    print(f'working on numors {numor_list}')
    tolerance_ccw=40 # default is for negative field (ccw rotation)
    tolerance_cw=5
    if '+' in title or 'Positive' in title:
        tolerance_ccw, tolerance_cw = tolerance_cw, tolerance_ccw
        print('positive field')
    for numor in numor_list:
        # Load the numor data using np.load()
        data = np.load(path_to_npzs + numor +'.npz')['data']
        # remove bad DPs from numor
        for index in filtered_indices[numor]:
            print(f'deleted index {index} from {numor}')
            data = np.delete(data, index, axis=0)
        dps = mask_and_blur_images(data)
        # Run grid_search_offset function with the loaded data


        offset1, offset2 = grid_search_offset(dps, offset_step=0.5, tolerance_ccw=tolerance_ccw, tolerance_cw=tolerance_cw, 
                                        threshold_offset1=jump_threshold_offset1, jump_threshold_offset2=jump_threshold_offset2)
        np.savez(f"{numor}_ratchet_offsets.npz", offset1=offset1, offset2=offset2)
        # Append the result for plotting
        all_offsets.append((numor, offset1, offset2))
        print(f'finished numor {numor}')
    print('Next!')
    # Plot all numors for the current key
    for i, (numor, offset1, offset2) in enumerate(all_offsets):
        plt.plot(offset1-offset1[0], label='offset1 '+numor, color=colors[i])
        plt.plot(offset2-offset2[0], color=colors[i], linestyle='dotted')
    plt.title(f"{title}")
    plt.xlabel("index")
    plt.ylabel("Offset (degrees)")
    plt.legend()
    plt.grid(True)
    # plt.savefig(f'data/{title}_ratchet_model.png')
    plt.show()





## Plot for comparison

In [ ]:
# load hand clicked
seed_path = "/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/"
analysis_path = "/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/"

numor = '46349'
# Define file paths
cm_offset1_path = f"{seed_path}cm_{numor}_offset1_seed.npz"
cm_offset2_path = f"{seed_path}cm_{numor}_offset2_seed.npz"
yc_offset1_path = f"{seed_path}yc_{numor}_offset1_seed.npz"
yc_offset2_path = f"{seed_path}yc_{numor}_offset2_seed.npz"
dr_offset1_path = f"{seed_path}dr_{numor}_offset1_seed.npz"
dr_offset2_path = f"{seed_path}dr_{numor}_offset2_seed.npz"
nc_offset1_path = f"{seed_path}nc_{numor}_offset1_seed.npz"
nc_offset2_path = f"{seed_path}nc_{numor}_offset2_seed.npz"
oob_offset1_path = f"{analysis_path}{numor}_offsets.npz"
oob_offset2_path = f"{analysis_path}{numor}_offsets.npz"

# Load data
offset1cm = np.load(cm_offset1_path)['data']
offset2cm = np.load(cm_offset2_path)['data']
offset1yc = np.load(yc_offset1_path)['data']
offset2yc = np.load(yc_offset2_path)['data']
offset1dr = np.load(dr_offset1_path)['data']
offset2dr = np.load(dr_offset2_path)['data']
offset1nc = np.load(nc_offset1_path)['data']
offset2nc = np.load(nc_offset2_path)['data']
offset1oob = np.load(oob_offset1_path)['offset1']
offset2oob = np.load(oob_offset2_path)['offset2']

window_length = 5
polyorder=4
def compute_smoothed_derivative(offset, window_length=window_length, polyorder=polyorder):
    '''compute velocity of data after savgol_filter is applied, assumes frame rate of 10s'''
    smoothed_angle = savgol_filter(offset, window_length=window_length, polyorder=polyorder)
    time = (np.arange(offset.shape[0])+1)*10
    smoothed_derivative = (np.gradient(smoothed_angle, time))
    return smoothed_derivative

colors = {
    "cm": "blue",
    "yc": "red",
    "dr": "green",
    "nc": "purple",
    "oob": 'orange',
    'ratchet': 'hotpink'
}

# plot over different tolerances
tols = [5]#, 7, 6, 5, 4, 3]
plt.figure(figsize=(7,7))
for i, tol in enumerate(tols):
    viri = [plt.cm.viridis(i / len(tols)) for i in range(len(tols))]
    offset1r = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/ratchet_model/data/reverse_tol_{tol}_{numor}_offsets.npz')['offset1']
    offset2r = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/ratchet_model/data/reverse_tol_{tol}_{numor}_offsets.npz')['offset2']
    plt.plot(-(offset1r - offset1r[0]), color=viri[i], label=f'offset1 ratchet tol={tol}')
    # velo2 = compute_smoothed_derivative(offset2r)
    # velo = compute_smoothed_derivative(offset1r)
    # plt.plot(velo, label=f'velo {numor}')
    # plt.plot(velo2, linestyle=':')
    plt.plot(-(offset2r - offset2r[0]), color=viri[i], linestyle=':')

plt.plot(offset1-offset1[0], color= colors['ratchet'])
plt.plot(offset2-offset2[0], color=colors['ratchet'], linestyle=':')
# plt.plot(offset1oob - offset1oob[0], color=colors['oob'], label='offset1 GD')
# # plt.plot(offset2oob - offset2oob[0], color=colors['oob'], linestyle=':')
# plt.plot(offset1cm - offset1cm[0], label='offset1 cm', color=colors['cm'])
# # plt.plot(offset2cm - offset2cm[0], color=colors['cm'], linestyle=':')
# plt.plot(offset1yc - offset1yc[0], label='offset1 yc', color=colors["yc"])
# # plt.plot(offset2yc - offset2yc[0], color=colors["yc"], linestyle=':')
# plt.plot(offset1dr - offset1dr[0], label='offset1 dr', color=colors["dr"])
# # plt.plot(offset2dr - offset2dr[0], color=colors["dr"], linestyle=':')
# plt.plot(offset1nc-offset1nc[0], label=f'offset1 nc', color=colors["nc"])
# # plt.plot(offset2nc-offset2nc[0], color=colors["nc"], linestyle=':')

plt.grid(True)
# plt.ylabel('offset')
plt.ylabel('angular velo deg/sec')
plt.xlabel('index')
plt.legend()
plt.title(f'{movie_to_title[str(numor)]}')
plt.show()


# # plot over different numors
# tol = 5
# numor_list = ['46349', '56059', '54261', '69958']
# plt.figure(figsize=(15,5))
# for i, numor in enumerate(numor_list):
#     viri = [plt.cm.viridis(i / len(numor_list)) for i in range(len(numor_list))]
#     offset1r = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/ratchet_model/data/reverse_tol_{tol}_{numor}_offsets.npz')['offset1']
#     offset2r = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/ratchet_model/data/reverse_tol_{tol}_{numor}_offsets.npz')['offset2']
#     # plt.plot(-(offset1r - offset1r[0]), color=viri[i], label=f'offset1 ratchet tol={tol}')
#     # velo2 = compute_smoothed_derivative(offset2r)
#     # velo = compute_smoothed_derivative(offset1r)
#     # plt.plot(velo, color=viri[i], label=f'velo {numor}')
#     # plt.axhline(0, color='black')
#     # plt.plot(velo2, color=viri[i]linestyle=':')
#     # plt.plot(-(offset2r - offset2r[0]), color=viri[i], linestyle=':')

# plt.grid(True)
# plt.ylabel('angular velo deg/sec')
# plt.xlabel('index')
# plt.legend()
# plt.title(f'{numor_list}, window length={window_length}, poly order={polyorder}')
# plt.show()




# --MISC FUNCTION BELOW--

## Use GD to tune ratchet model

In [ ]:
import torch
MAX_ITER_OFFSET1 = 50
MAX_ITER_OFFSET2 = 50
LR = 1e-2
GD_numor = '44192'
offset1_init = torch.tensor(0., requires_grad=True)
offset2_init = torch.tensor(0., requires_grad=True)
dps_seed = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/npz_sept_numor_data/{numor}.npz')['data']

offset1_seed_array = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/ratchet_model/data/{GD_numor}_ratchet_offsets.npz')['offset1']
offset2_seed_array = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/ratchet_model/data/{GD_numor}_ratchet_offsets.npz')['offset2']
i = 0
offset1_seed = offset1_seed_array[i]
offset2_seed = offset2_seed_array[i]

def filter_function_torch(k, offset, n_folds=n_folds):
    filter = torch.exp(k * torch.log((torch.cos(n_folds / 2 * (torch.tensor(DATA_THETA) + offset)))**2))
    # print('k=', k, 'n_folds=', n_folds)
    # plt.imshow(filter)
    # plt.title(f'n_folds={n_folds}, k={k}')
    # plt.show()
    return filter

def gradient_descent_optimize_seeded_offset(intensity, offset1_seed, offset2_seed):
    '''
    Takes in offset seeds from ratchet model and refines the offset.

    all offset inputs [=] degrees
    all offset outputs [=] degrees
    '''
    # initialize the offsets with the seeds from ratchet model
    offset1 = torch.tensor(np.deg2rad(offset1_seed), dtype=torch.float32, requires_grad=True)
    offset2 = torch.tensor(np.deg2rad(offset2_seed), dtype=torch.float32, requires_grad=True)

    opt1 = torch.optim.Adam([offset1], lr=LR)
    opt2 = torch.optim.Adam([offset2], lr=LR)

    for _ in range(MAX_ITER_OFFSET1):
        loss = -(torch.tensor(intensity, dtype=torch.float32) * filter_function_torch(k, offset1)).sum()
        opt1.zero_grad()
        loss.backward()
        opt1.step()

    for _ in range(MAX_ITER_OFFSET2):
        loss = -(torch.tensor(intensity, dtype=torch.float32) * filter_function_torch(k, offset2)).sum()
        opt2.zero_grad()
        loss.backward()
        opt2.step()
    offset1 = np.rad2deg(offset1.detach().item())
    offset2 = np.rad2deg(offset2.detach().item())
    return offset1, offset2

# off1, off2 = gradient_descent_optimize_seeded_offset(dps[i])
# print(offset1_seed, offset2_seed, off1, off2)

refined_offset1, refined_offset2 = [], []

for i, dp in enumerate(dps_seed):
    off1, off2 = gradient_descent_optimize_seeded_offset(dp, offset1_seed_array[i], offset2_seed_array[i])
    refined_offset1.append(off1)
    refined_offset2.append(off2)